In [1]:
#import
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import string
import re

In [52]:
#read bad-words.csv
df = pd.read_csv('C:\\Users\\HTY\\Desktop\\cs196\\bad-words.csv')
BWlist = df['BW'].values.tolist()

In [53]:
#set BasicFilter:
class BasicFilter():
    def __init__(self):
        self.keywords = set(BWlist)

    def filter(self,message,repl="*"):
        message = str(message).lower()
        for kw in self.keywords:
            message = message.replace(kw, repl)
        return message

In [54]:
#try basic filter
print(BasicFilter().filter('fuck u'))

* u


In [4]:
#import train data
train_df = pd.read_csv('C:\\Users\\HTY\\Desktop\\cs196\\train.csv')
train_df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [5]:
train_df['sum'] = train_df['toxic'] + train_df['severe_toxic'] + train_df['obscene'] + train_df['threat'] + train_df['insult'] + train_df['identity_hate']

In [17]:
train_df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,sum,label
0,0000997932d777bf,explanation why the edits made under my userna...,0,0,0,0,0,0,0,False
1,000103f0d9cfb60f,d aww he matches this background colour i am s...,0,0,0,0,0,0,0,False
2,000113f07ec002fd,hey man i am really not trying to edit war it ...,0,0,0,0,0,0,0,False
3,0001b41b1c6bb37e,more i cannot make any real suggestions on imp...,0,0,0,0,0,0,0,False
4,0001d958c54c6e35,you sir are my hero any chance you remember wh...,0,0,0,0,0,0,0,False


In [7]:
train_df['label'] = train_df['sum'] >= 1

In [32]:
train_df.head(10)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,sum,label
0,0000997932d777bf,explanation why the edits made under my userna...,0,0,0,0,0,0,0,False
1,000103f0d9cfb60f,d aww he matches this background colour i am s...,0,0,0,0,0,0,0,False
2,000113f07ec002fd,hey man i am really not trying to edit war it ...,0,0,0,0,0,0,0,False
3,0001b41b1c6bb37e,more i cannot make any real suggestions on imp...,0,0,0,0,0,0,0,False
4,0001d958c54c6e35,you sir are my hero any chance you remember wh...,0,0,0,0,0,0,0,False
5,00025465d4725e87,congratulations from me as well use the tools ...,0,0,0,0,0,0,0,False
6,0002bcb3da6cb337,cocksucker before you piss around on my work,1,1,1,0,1,0,4,True
7,00031b1e95af7921,your vandalism to the matt shirvington article...,0,0,0,0,0,0,0,False
8,00037261f536c51d,sorry if the word nonsense was offensive to yo...,0,0,0,0,0,0,0,False
9,00040093b2687caa,alignment on this subject and which are contra...,0,0,0,0,0,0,0,False


In [47]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.externals import joblib

In [57]:
class AdvancedFilter():
    
    def __init__(self) -> None:
        pass

    def clean_text(self, text):
        text = text.lower()
        text = re.sub(r"what's", "what is ", text)
        text = re.sub(r"\'s", " ", text)
        text = re.sub(r"\'ve", " have ", text)
        text = re.sub(r"can't", "cannot ", text)
        text = re.sub(r"n't", " not ", text)
        text = re.sub(r"i'm", "i am ", text)
        text = re.sub(r"\'re", " are ", text)
        text = re.sub(r"\'d", " would ", text)
        text = re.sub(r"\'ll", " will ", text)
        text = re.sub(r"\'scuse", " excuse ", text)
        text = re.sub('\W', ' ', text)
        text = re.sub('\s+', ' ', text)
        text = text.strip(' ')
        return text
    
    def vectorization(self, train_df):
        vect = TfidfVectorizer(max_features=5000,stop_words='english')
        train_df['comment_text'] = train_df['comment_text'].map(lambda com : self.clean_text(com))
        X = train_df.comment_text
        vect.fit(X)
        joblib.dump(vect, 'vect.pkl')
        return vect
    
    def train_model(self, train_df):
        X = train_df.comment_text
        vect = TfidfVectorizer(max_features=5000,stop_words='english')
        X_dtm = vect.fit_transform(X)
        logreg = LogisticRegression(C=12.0)
        y = train_df['label']
        logreg.fit(X_dtm, y)
        joblib.dump(logreg, 'lr.pkl')
        return logreg
       
    def ufilter(self, message):
        clean = self.clean_text(message)
        data = [[clean]]
        df = pd.DataFrame(data, columns=['comment_text'])
        X_test = df.comment_text
        vect = joblib.load('vect.pkl')
        test = vect.transform(X_test)
        model = joblib.load('lr.pkl')
        return model.predict(test)

    def filter(self, message):
        if self.ufilter(message):
            return ""
        return message



In [58]:
print(AdvancedFilter().filter('It aims for young people all race. It sold fast food that all Americans know. The purpose of it is make all McDonalds impressive, including the historical successful brand. It has some pop music element. The music makes audience enjoyable. The mood is happy. It also includes the short time cost from the restaurant to customer’s hands. It use hyperbole to make the food looks more beautiful and delicious.'))

It aims for young people all race. It sold fast food that all Americans know. The purpose of it is make all McDonalds impressive, including the historical successful brand. It has some pop music element. The music makes audience enjoyable. The mood is happy. It also includes the short time cost from the restaurant to customer’s hands. It use hyperbole to make the food looks more beautiful and delicious.
